In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [ ]:
def sort_events(indices, similarities):
    results = list(dict(sorted(dict(zip(indices, similarities)).items(), key=lambda item: item[1], reverse=True)).keys())
    return results

def filter_events(indices, similarities, threshold):
    results = sorted(dict(zip(indices, similarities)).items(), key=lambda item: item[1], reverse=True)
    results = list(dict(filter(lambda x: x[1] > threshold, results)).keys())
    return results

In [ ]:
events = {
    1: 'К 140-летию со дня рождения Игоря Стравинского',
    2: 'Тренировки в Спортивном парке',
    3: 'Закрытие Большого летнего музыкального фестиваля «Сириус»',
    4: 'Вечер серенад с «Виртуозами Москвы»',
    5: 'Музыкальная сборная Россия. Фортепианный марафон',
    6: 'Религиозные мотивы в советской постмодернистской литературе',
    7: 'Мастер-класс по гравюре на офортном станке',
    8: 'Дискуссия «Между мягкой обложкой и коллекционным изданием»',
    9: 'Лекция и презентация книги «Древнерусская криптография»',
    10: 'Путешествие с Запада на Восток',
    11: 'Виктор Цой и группа «Кино». Фотографии. Воспоминания. Презентация книги',
    12: 'Кинопоказ в Мурино',
    13: '«КАРМЕН». ТЕАТРАЛЬНЫЙ КОНЦЕРТ', 
    14: '«ГОЛОСА РУССКОГО ОРКЕСТРА». КОНЦЕРТ АНДРЕЕВСКОГО ОРКЕСТРА Дирижер – Дмитрий Хохлов',
    15: '«ТАНГО И ДЖАЗ»',
    16: 'ПРИНОШЕНИЕ ИЛЬЕ МУСИНУ. ОРКЕСТР КОСТРОМСКОЙ ФИЛАРМОНИИ', 
    17: 'АНСАМБЛЬ ДОНСКИХ КАЗАКОВ ИМ. АНАТОЛИЯ КВАСОВА',
    18: 'ОТКРЫТИЕ КОНЦЕРТНОГО СЕЗОНА. ХОР И ОРКЕСТР КАПЕЛЛЫ',
    19: 'МИР Российская Премьер-Лига. Тур 8. "Краснодар" - "Сочи". ',
    20: 'Футбол. Лига чемпионов. "Аякс" (Нидерланды) - "Рейнджерс" (Шотландия). '
}
sentences = list(events.values())
indices = list(events.keys())
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(20, 512)

## Search engine prototype

In [ ]:
search_query = 'спорт'
query_embedding = model.encode(search_query)
query_embedding.shape
similarities = cosine_similarity(
    [query_embedding],
    sentence_embeddings
).tolist()[0]
similarities

[-0.028517695143818855,
 0.5193768739700317,
 -0.0030378776136785746,
 0.003912292420864105,
 0.22077743709087372,
 0.12841518223285675,
 -0.0644831508398056,
 -0.10213816165924072,
 0.012773217633366585,
 0.05484943836927414,
 -0.012179090641438961,
 0.10704102367162704,
 0.11932618916034698,
 -0.026126617565751076,
 0.1474267840385437,
 0.05623435229063034,
 0.0215598251670599,
 0.014595860615372658,
 0.003733782097697258,
 0.22167354822158813]

In [ ]:
sort_events(indices, similarities)

[2, 20, 5, 15, 6, 13, 12, 16, 10, 17, 18, 9, 4, 19, 3, 11, 14, 1, 7, 8]

In [ ]:
filter_events(indices, similarities, threshold=0.2)

[2, 20, 5]

In [ ]:
print(*[events[id] for id in filter_events(indices, similarities, threshold=0.2)], sep='\n')

Тренировки в Спортивном парке
Футбол. Лига чемпионов. "Аякс" (Нидерланды) - "Рейнджерс" (Шотландия). 
Музыкальная сборная Россия. Фортепианный марафон


## RecSys prototype

In [ ]:
history = []
seen_indices = range(1, 11)
for idx in seen_indices:
    history.append(model.encode(sentences[idx]))

In [ ]:
def range_events(events, indices, seen_indices):
    history = []
    seen_indices = range(1, 11)
    for idx in seen_indices:
        history.append(model.encode(sentences[idx]))
    similarities = np.array(np.mean([(cosine_similarity([emb], sentence_embeddings) * ratio).tolist()[0] 
                                    for emb, ratio in zip(history, np.linspace(0.5, 1.5, len(history)))], axis=0))
    result = sort_events(indices, similarities)
    return result

In [ ]:
def id_to_text(events, idx_list):
    return [events[id] for id in sort_events(indices, similarities)]

In [ ]:
ranking = range_events(events, indices, seen_indices)
id_to_text(events, ranking)

['Лекция и презентация книги «Древнерусская криптография»',
 'Дискуссия «Между мягкой обложкой и коллекционным изданием»',
 'Виктор Цой и группа «Кино». Фотографии. Воспоминания. Презентация книги',
 'Вечер серенад с «Виртуозами Москвы»',
 'Закрытие Большого летнего музыкального фестиваля «Сириус»',
 'Религиозные мотивы в советской постмодернистской литературе',
 'Путешествие с Запада на Восток',
 'Музыкальная сборная Россия. Фортепианный марафон',
 '«ТАНГО И ДЖАЗ»',
 'Мастер-класс по гравюре на офортном станке',
 'ПРИНОШЕНИЕ ИЛЬЕ МУСИНУ. ОРКЕСТР КОСТРОМСКОЙ ФИЛАРМОНИИ',
 '«ГОЛОСА РУССКОГО ОРКЕСТРА». КОНЦЕРТ АНДРЕЕВСКОГО ОРКЕСТРА Дирижер – Дмитрий Хохлов',
 '«КАРМЕН». ТЕАТРАЛЬНЫЙ КОНЦЕРТ',
 'Кинопоказ в Мурино',
 'ОТКРЫТИЕ КОНЦЕРТНОГО СЕЗОНА. ХОР И ОРКЕСТР КАПЕЛЛЫ',
 'МИР Российская Премьер-Лига. Тур 8. "Краснодар" - "Сочи". ',
 'АНСАМБЛЬ ДОНСКИХ КАЗАКОВ ИМ. АНАТОЛИЯ КВАСОВА',
 'К 140-летию со дня рождения Игоря Стравинского',
 'Тренировки в Спортивном парке',
 'Футбол. Лига чемпионов.

In [ ]:
similarities = np.array(np.mean([(cosine_similarity([emb], sentence_embeddings) * ratio).tolist()[0] 
                                 for emb, ratio in zip(history, np.linspace(0.5, 1.5, len(history)))], axis=0))

In [ ]:
print(*[events[id] for id in sort_events(indices, similarities)], sep='\n')

Лекция и презентация книги «Древнерусская криптография»
Дискуссия «Между мягкой обложкой и коллекционным изданием»
Виктор Цой и группа «Кино». Фотографии. Воспоминания. Презентация книги
Вечер серенад с «Виртуозами Москвы»
Закрытие Большого летнего музыкального фестиваля «Сириус»
Религиозные мотивы в советской постмодернистской литературе
Путешествие с Запада на Восток
Музыкальная сборная Россия. Фортепианный марафон
«ТАНГО И ДЖАЗ»
Мастер-класс по гравюре на офортном станке
ПРИНОШЕНИЕ ИЛЬЕ МУСИНУ. ОРКЕСТР КОСТРОМСКОЙ ФИЛАРМОНИИ
«ГОЛОСА РУССКОГО ОРКЕСТРА». КОНЦЕРТ АНДРЕЕВСКОГО ОРКЕСТРА Дирижер – Дмитрий Хохлов
«КАРМЕН». ТЕАТРАЛЬНЫЙ КОНЦЕРТ
Кинопоказ в Мурино
ОТКРЫТИЕ КОНЦЕРТНОГО СЕЗОНА. ХОР И ОРКЕСТР КАПЕЛЛЫ
МИР Российская Премьер-Лига. Тур 8. "Краснодар" - "Сочи". 
АНСАМБЛЬ ДОНСКИХ КАЗАКОВ ИМ. АНАТОЛИЯ КВАСОВА
К 140-летию со дня рождения Игоря Стравинского
Тренировки в Спортивном парке
Футбол. Лига чемпионов. "Аякс" (Нидерланды) - "Рейнджерс" (Шотландия). 
